In [1]:
import sys
sys.path.insert(0,'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence')

In [2]:
from lenskit.metrics import predict
import pandas as pd
import matplotlib.pyplot as plt

from lenskit.batch_ca import MultiEval
from lenskit.algorithms_ca import user_knn_ca, Recommender
from lenskit.metrics.predict import rmse
from lenskit import datasets
from lenskit.datasets import MovieLens
from lenskit import crossfold as xf
from lenskit import topn, datasets, batch_ca # *not* batch 
from lenskit import topn, util #, metrics
from lenskit.crossfold import partition_users, SampleN, partition_rows

In [3]:
util.log_to_notebook()

[   INFO] lenskit.util.log notebook logging configured


In [4]:
data = 'jester' # 'ml10m'

In [5]:
path = 'D:\\results_conf_aware_nbhd_KDD2021\\' + data + '\\baselines\\'

In [6]:
list_lengths = list((1,2,3,4,5,6,7,8,9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100))

In [ ]:
for list_length in list_lengths:

    print(list_length)

    runs = pd.read_csv(path + 'runs.csv')
    runs.set_index('RunId', inplace = True)
    #runs.head()

    recs = pd.read_parquet(path + 'recommendations.parquet')
    #del recs['RunId']
    #recs.head()

    recs = recs[['item', 'score', 'user','rank','RunId']]
    #recs.head()

    recs = recs[recs['rank'] <= list_length]

    truth = pd.read_csv(path + 'truth.csv')
    #truth.head()

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg) # precision, recall, recip_rank, dcg, ndcg
    rla.add_metric(topn.precision)
    raw_ndcg = rla.compute(recs, truth)
    #raw_ndcg.head()

    ### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
    ndcg = raw_ndcg.join(runs[['name', 'nnbrs']], on = 'RunId')
    #ndcg.head()

    ### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
    ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()

    ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean().to_csv(path + 'results' + str(eval('list_length')) + '.csv')

1
[   INFO] lenskit.topn analyzing 587368 recommendations (367105 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
